In [1]:
from gensim import models

In [2]:
# 불러들이는데 시간 오지게 걸리고, 다운 받아서 쥬피터 폴더에 넣어야함
ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [3]:
ko_model?

In [3]:
# pre-trained된 단어-vector set, 개를 치면 이런 벡터가 나옴
ko_model.wv.word_vec('개') # 이런식으로 vectorize 됩니다

array([ 7.83980787e-02,  9.76187885e-02, -7.85277858e-02,  1.01604983e-01,
       -7.33330399e-02,  1.99361414e-01,  2.87973527e-02, -4.82029244e-02,
       -6.48322031e-02,  1.31404757e-01, -1.08594797e-03, -4.06343453e-02,
        6.77537024e-02, -6.27582669e-02,  1.47260278e-02, -1.36697933e-01,
        1.01806007e-01, -9.05955508e-02,  7.79545978e-02,  9.42295417e-02,
       -2.81342454e-02,  2.83462517e-02, -7.69589795e-03,  6.88294172e-02,
        5.99902831e-02,  6.13487720e-01,  8.88875201e-02, -8.06730313e-05,
       -3.28737386e-02, -6.24504425e-02,  1.70042858e-01, -4.16966788e-02,
        3.41004133e-02,  1.17442183e-01, -7.60835875e-03,  6.93268925e-02,
        2.40220856e-02, -9.32281092e-02,  1.95473403e-01, -2.47712776e-01,
        6.97853640e-02, -3.70084345e-02, -2.98745465e-03, -2.62751821e-02,
        7.47143850e-02, -1.84808403e-01,  8.72078240e-02, -1.26332879e-01,
       -4.01524305e-02, -1.19021617e-01,  1.73295103e-02,  9.39118713e-02,
        8.70536715e-02, -

In [10]:
from konlpy.tag import Okt # 이걸 써서 포스 태깅을 할꺼고
okt = Okt()

In [5]:
import pandas as pd # 이것들 가져와
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [43]:
train_data.head() # 네이버 리뷰 트레인 데이터는 이렇게 생겼다

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [89]:
label = train_data['label']

train_label = np.array(label[:100])

In [90]:
train_label

array([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1], dtype=int64)

In [8]:
sentences = train_data['document'].tolist() # 일단 저는 리스트로 했고

In [ ]:
# 단어 토큰화 하기
def tokenize(sentence):
    okt = Okt()
    tokenized_sentence = []

    # 우선 단어의 기본형으로 모두 살리고, 명사, 동사, 영어만 담는다.
    # 그냥 nouns로 분리하는 것보다 좀 더 정확하고 많은 데이터를 얻을 수 있다.
    for line in sentence:
        result = []
        temp_sentence = okt.pos(line, norm=True, stem=True) # 먼저 형태소 분리해서 리스트에 담고
        print(temp_sentence)
        for i in temp_sentence:                             
            if (i[1] == 'Noun' or i[1] == 'Adjective' or i[1] == 'Alpha'):                  
                result.append(i[0])
            
        tokenized_sentence.append(result)

    return tokenized_sentence

In [91]:
toke = tokenize(sentences[:100]) # 이런식으로 프린팅 되지만 결과는 다름

[('아', 'Exclamation'), ('더빙', 'Noun'), ('..', 'Punctuation'), ('진짜', 'Noun'), ('짜증나다', 'Adjective'), ('목소리', 'Noun')]
[('흠', 'Noun'), ('...', 'Punctuation'), ('포스터', 'Noun'), ('보고', 'Noun'), ('초딩', 'Noun'), ('영화', 'Noun'), ('줄', 'Noun'), ('....', 'Punctuation'), ('오버', 'Noun'), ('연기', 'Noun'), ('조차', 'Josa'), ('가볍다', 'Adjective'), ('않다', 'Verb')]
[('너', 'Modifier'), ('무재', 'Noun'), ('밓었', 'Noun'), ('다그', 'Noun'), ('래서', 'Noun'), ('보다', 'Verb'), ('추천', 'Noun'), ('한', 'Josa'), ('다', 'Adverb')]
[('교도소', 'Noun'), ('이야기', 'Noun'), ('구먼', 'Noun'), ('..', 'Punctuation'), ('솔직하다', 'Adjective'), ('재미', 'Noun'), ('는', 'Josa'), ('없다', 'Adjective'), ('..', 'Punctuation'), ('평점', 'Noun'), ('조정', 'Noun')]
[('사이', 'Modifier'), ('몬페', 'Noun'), ('그', 'Determiner'), ('의', 'Noun'), ('익살스럽다', 'Adjective'), ('연기', 'Noun'), ('가', 'Josa'), ('돋보이다', 'Verb'), ('영화', 'Noun'), ('!', 'Punctuation'), ('스파이더맨', 'Noun'), ('에서', 'Josa'), ('늙다', 'Verb'), ('보이다', 'Verb'), ('하다', 'Verb'), ('커스틴', 'Noun'), ('던스트', 'Noun'

[('리얼리티', 'Noun'), ('가', 'Josa'), ('뛰어나다', 'Adjective'), ('한데', 'Eomi'), ('크다', 'Verb'), ('공감', 'Noun'), ('은', 'Josa'), ('안', 'Noun'), ('간다', 'Noun'), ('.', 'Punctuation'), ('이민기', 'Noun'), ('캐릭터', 'Noun'), ('는', 'Josa'), ('정신의학', 'Noun'), ('상', 'Suffix'), ('분노조절', 'Noun'), ('장애', 'Noun'), ('초기', 'Noun'), ('증상', 'Noun'), ('이다', 'Verb'), ('.', 'Punctuation'), ('툭하면', 'Noun'), ('사람', 'Noun'), ('패', 'Noun'), ('고', 'Josa'), ('욕', 'Noun'), ('하고', 'Josa'), ('물건', 'Noun'), ('파손', 'Noun'), ('하고', 'Josa'), ('..', 'Punctuation'), ('조금', 'Noun'), ('오', 'Modifier'), ('바', 'Noun'), ('이다', 'Verb'), ('.', 'Punctuation'), ('극', 'Modifier'), ('초반', 'Noun'), ('엔', 'Josa'), ('신선하다', 'Adjective'), ('가면', 'Noun'), ('갈수록', 'Noun'), ('이민기', 'Noun'), ('정신', 'Noun'), ('상태', 'Noun'), ('공감', 'Noun'), ('불가', 'Noun'), ('.', 'Punctuation')]
[('마이너스', 'Noun'), ('는', 'Josa'), ('왜', 'Noun'), ('없다', 'Adjective'), ('ㅋ', 'KoreanParticle'), ('뮤비', 'Noun'), ('보고', 'Noun'), ('영화', 'Noun'), ('수준', 'Noun'), ('딱', 'Adverb'), (

[('완전', 'Noun'), ('재밌다', 'Adjective'), ('ㅋㅋㅋ', 'KoreanParticle'), ('백', 'Modifier'), ('인공', 'Noun'), ('주귀', 'Noun'), ('여', 'Josa'), ('움', 'Noun'), ('ㅋㅋㅋ', 'KoreanParticle')]
[('인상', 'Noun'), ('적', 'Suffix'), ('인', 'Josa'), ('영화', 'Noun'), ('이다', 'Verb')]
[('어내스트', 'Noun'), ('와', 'Josa'), ('셀레스틴', 'Noun'), ('완전', 'Noun'), ('강추', 'Noun'), ('에요', 'Josa'), ('~', 'Punctuation'), ('정말', 'Noun'), ('재밌다', 'Adjective'), ('^^', 'Punctuation')]
[('재미있다', 'Adjective'), ('영화', 'Noun'), ('이다', 'Adjective'), ('.', 'Punctuation')]
[('클라라', 'Noun'), ('볼라', 'Noun'), ('고', 'Josa'), ('화신', 'Noun'), ('보다', 'Verb'), ('아니다', 'Adjective')]
[('진짜', 'Noun'), ('보다', 'Verb'), ('너무', 'Adverb'), ('슬프다', 'Adjective'), ('영화', 'Noun'), ('다', 'Josa')]
[('설정', 'Noun'), ('이', 'Josa'), ('재밌다', 'Adjective'), ('새롭다', 'Adjective'), ('에피소드', 'Noun'), ('내', 'Noun'), ('에서', 'Josa'), ('메인', 'Noun'), ('스토리', 'Noun'), ('도', 'Josa'), ('차차', 'Noun'), ('나오다', 'Verb'), ('재밌다', 'Adjective')]


In [92]:
toke # 이제 각 문장의 길이를 맞춰주기 위해 padding을 넣고

[['더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '가볍다'],
 ['무재', '밓었', '다그', '래서', '추천'],
 ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정'],
 ['몬페', '의', '익살스럽다', '연기', '영화', '스파이더맨', '커스틴', '던스트', '이쁘다'],
 ['막', '걸음', '마', '세', '초등학교', '학년', '생인', '영화', '반개', '아깝다', '움'],
 ['원작', '긴장감', '제대로'],
 ['별',
  '반개',
  '아깝다',
  '욕',
  '이응경',
  '길용우',
  '생활',
  '년',
  '정말',
  '발',
  '해도',
  '그것',
  '납치',
  '감금',
  '반복',
  '반복',
  '드라마',
  '가족',
  '없다',
  '연기',
  '사람',
  '네'],
 ['액션', '없다', '재미', '있다', '몇', '안되다', '영화'],
 ['왜', '평점', '낮다', '꽤', '볼', '데', '헐리우드', '화려하다', '있다'],
 ['인피니트', '짱', '진짜'],
 ['볼때', '향수', '자극', '허진호', '감성', '절제', '멜로', '달인'],
 ['울면', '손', '횡단보도', '때', '뻔', '이범수', '연기', '드럽다'],
 ['담백하다', '깔끔하다', '좋다', '로만', '자꾸', '그', '사람', '것'],
 ['취향',
  '존중',
  '다지',
  '진짜',
  '내생',
  '극장',
  '영화',
  '가장',
  '노잼',
  '노',
  '감동',
  '임',
  '스토리',
  '어거지',
  '감동',
  '어거지'],
 ['냥', '매번', '긴장', '재밌다'],
 ['사람', '바스코', '이기', '락스', '코', '바비', '이기', '아이돌', '그냥'

In [93]:
def pad_sequence(sentences, padding_word="<PAD/>"): #  오른쪽을 패딩주기
    maxlen = max(len(x) for x in sentences)
    padded_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        num_padding = maxlen - len(sentence)
        new_sentence = sentence + [padding_word] * num_padding
        padded_sentences.append(new_sentence)
    return padded_sentences

In [95]:
padded = pad_sequence(toke)

In [102]:
padded

[['더빙',
  '진짜',
  '짜증나다',
  '목소리',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>'],
 ['흠',
  '포스터',
  '보고',
  '초딩',
  '영화',
  '줄',
  '오버',
  '연기',
  '가볍다',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>'],
 ['무재',
  '밓었',
  '다그',
  '래서',
  '추천',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',
  '<PAD/>',

In [97]:
len(padded)

100

In [28]:
import numpy as np

In [98]:
# ko_model.wv.word_vec('<PAD/>')
paddedarray = np.array([ ko_model.wv.word_vec(token) for x in padded for token in x])

In [ ]:
paddedarray.shape # 이런 형태네? max_length of sent = 22(34),근데 10개의 sent, 22*10 fasttext = 300(vector) 

In [99]:
pading = [ ko_model.wv.word_vec(token) for x in padded for token in x]
len(pading)

3400

In [104]:
pading# 리스트 안에 220개 들어가있어, 22개를 하나로 합쳐줘야해, 그래서 10차원 만들어야 해 ( 10개 기준)
      # 100개로 해보니,, 34로 해야함,                  (100차원) 100개 기준

def divide_list(l, n): 
    # 리스트 l의 길이가 n이면 계속 반복
    for i in range(0, len(l), n): 
        yield l[i:i + n] 
# 한 리스트에 몇개씩 담을지 결정
n = 3
result = list(divide_list(pading, 34))


    

In [105]:
final_array = np.array(result)

In [107]:
final_array.shape

(100, 34, 300)

In [31]:
import tensorflow as tf
im

In [34]:
from tensorflow import keras

In [108]:
embedding_dim = 200
filter_sizes = (3, 4, 5)
num_filters = 100
dropout = 0.5
hidden_dims = 100

batch_size = 50
num_epochs = 10
min_word_count = 1
context = 10

conv_blocks = []

sequence_length = 200

# input_shape = (sequence_length, embedding_dim) # input shape for
input_shape = (34, 300) # input shape for data, (max_length of sent, vect)

model_input = keras.layers.Input(shape=input_shape)
z = model_input
for sz in filter_sizes:
    conv = keras.layers.Conv1D(filters=num_filters,
                         kernel_size=sz,
                         padding="valid",
                         activation="relu",
                         strides=1)(z)
    conv = keras.layers.MaxPooling1D(pool_size=2)(conv)
    conv = keras.layers.Flatten()(conv)
    conv_blocks.append(conv)
z = keras.layers.Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

z = keras.layers.Dropout(dropout)(z)
z = keras.layers.Dense(hidden_dims, activation="relu")(z)
model_output = keras.layers.Dense(1, activation="sigmoid")(z)

model = keras.Model(model_input, model_output)
model.compile(loss="binary_crossentropy", optimizer="adadelta", metrics=["accuracy"])


In [109]:
model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 34, 300)]    0                                            
__________________________________________________________________________________________________
conv1d_14 (Conv1D)              (None, 32, 100)      90100       input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 31, 100)      120100      input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, 30, 100)      150100      input_5[0][0]                    
_______________________________________________________________________________________

In [110]:
model.fit(final_array, train_label,epochs = 10)

Epoch 1/10
4/4 [==============================] - 0s 23ms/step - loss: 0.7055 - accuracy: 0.4500
Epoch 2/10
4/4 [==============================] - 0s 20ms/step - loss: 0.7001 - accuracy: 0.4600
Epoch 3/10
4/4 [==============================] - 0s 20ms/step - loss: 0.7045 - accuracy: 0.4600
Epoch 4/10
4/4 [==============================] - 0s 31ms/step - loss: 0.7012 - accuracy: 0.4700
Epoch 5/10
4/4 [==============================] - 0s 26ms/step - loss: 0.7089 - accuracy: 0.4400
Epoch 6/10
4/4 [==============================] - 0s 21ms/step - loss: 0.7034 - accuracy: 0.4600
Epoch 7/10
4/4 [==============================] - 0s 20ms/step - loss: 0.7060 - accuracy: 0.4700
Epoch 8/10
4/4 [==============================] - 0s 20ms/step - loss: 0.7102 - accuracy: 0.4500
Epoch 9/10
4/4 [==============================] - 0s 20ms/step - loss: 0.7010 - accuracy: 0.4600
Epoch 10/10
4/4 [==============================] - 0s 20ms/step - loss: 0.7108 - accuracy: 0.4500


In [82]:
final_array

(10, 22, 300)